<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Category_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [14]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 163kB 2.7MB/s 
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [16]:
%%time
# reading the data
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'category'].drop(columns = ['attribute_name'])
# combining similar category attribute values and removing spaces
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'sweatshirthoodie' if x == 'sweatshirt hoodie' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'blazerscoatsjackets' if x == 'blazer , coat jacket' else x)

CPU times: user 426 ms, sys: 60.6 ms, total: 487 ms
Wall time: 1.68 s


In [17]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
3,01DVCVGDJQC4J8JJ35PKZ4J169,loewe,hammock small textured leather shoulder bag,black textured leather calf hook fasten open w...,bag tote bag tote bag,accessory,item 's measurement depth 14 cm height 23 cm m...
32,01DPGSTG4M1RXB26QMMN0MPPB8,j crew,mockneck sweater supersoft yarn,cozy flattering mockneck sweater feel good loo...,sweater,sweater,NaN
36,01E2KKRH99SZ1PTR2E2ABVYJ5S,alo,vapor high waist legging,signature lift fabric moisture wicking technol...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."
50,01E2M1MYJ1FYVHG2AMXHDMNQSP,alo,prism high waist capris,high waistband help flat figure moisture wicki...,unknown,NaN,"xxs 00 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 hig..."
52,01E2KZ8BZF6M1XCHBZGEX3AF78,alo,cover tank,hard earn abs supersoft faux wrap tank 's crop...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."


In [0]:
# creating a feature combining brand, productname, description and brand category
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
# creating a function to create uni-grams and tokenizing data
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
# creating a function to create bi-grams and tokenizing data
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [22]:
Tokenizer()

In [0]:
# creating tf-idf vectors from uni-gram encoding
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
# creating tf-idf vectors from bi-gram encoding
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [46]:
X1.shape

(4194, 155)

In [0]:
# concatenating uni-gram and bi-gram
X = np.concatenate([X1, X2], axis = 1)

In [0]:
# splitting data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [49]:
# balancing the sample sets as the proportion of labels is unbalanced
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [0]:
num_classes = y.shape[1]

In [0]:
# defining LSTM model with sigmoid activation functions for prediction
def define_model():
    inputs = Input(shape=(length1+length2,))
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [55]:
model = define_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 310)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 310, 100)          615000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 310, 16)           7488      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 310, 7)            119       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 7)              15197     
_________________________________________________________________
reshape_3 (Reshape)          (None, 7)                 0         
_________________________________________________________________
activation_3 (Activation)    (None, 7)                 0   

In [56]:
# fitting data onto model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11143 samples, validate on 2786 samples
Epoch 1/10
11143/11143 [==============================] - 13s 1ms/step - loss: 0.4763 - accuracy: 0.8382 - val_loss: 0.6421 - val_accuracy: 0.8571
Epoch 2/10
11143/11143 [==============================] - 12s 1ms/step - loss: 0.3946 - accuracy: 0.8571 - val_loss: 0.7003 - val_accuracy: 0.8571
Epoch 3/10
11143/11143 [==============================] - 12s 1ms/step - loss: 0.3908 - accuracy: 0.8571 - val_loss: 0.7073 - val_accuracy: 0.8571
Epoch 4/10
11143/11143 [==============================] - 12s 1ms/step - loss: 0.3904 - accuracy: 0.8571 - val_loss: 0.7023 - val_accuracy: 0.8571
Epoch 5/10
11143/11143 [==============================] - 12s 1ms/step - loss: 0.3904 - accuracy: 0.8571 - val_loss: 0.6881 - val_accuracy: 0.8571
Epoch 6/10
11143/11143 [==============================] - 12s 1ms/step - loss: 0.3902 - accuracy: 0.8571 - val_loss: 0.6967 - val_accuracy: 0.8571
Epoch 7/10
11143/11143 [==============================] - 12s 1ms/ste

In [57]:
# evaluating model
model.evaluate(X_test, y_test)

420/420 [==============================] - 1s 2ms/step


[0.3093370903105963, 0.9153061509132385]

In [58]:
# #predictions for each class from predicting model
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

0    1869
3    1579
1     338
4     300
2     108
dtype: int64

In [59]:
# probability of predicting each class for various records
model.predict(X)

array([[0.23349158, 0.1403265 , 0.18513161, ..., 0.15322624, 0.09598885,
        0.01249978],
       [0.22711077, 0.21677308, 0.12408447, ..., 0.16336781, 0.09307244,
        0.01664122],
       [0.15537602, 0.27191517, 0.16586323, ..., 0.19173472, 0.08313783,
        0.01024285],
       ...,
       [0.2092981 , 0.18118417, 0.1617306 , ..., 0.15898196, 0.10170939,
        0.01042193],
       [0.43592936, 0.10214408, 0.1261948 , ..., 0.20925735, 0.10429128,
        0.02028095],
       [0.1647742 , 0.16356297, 0.18262945, ..., 0.16173075, 0.09111876,
        0.00940173]], dtype=float32)

In [60]:
# split of classes
pd.Series(y.argmax(axis = -1)).value_counts()

0    2184
4     672
2     427
1     353
5     295
6     146
3     117
dtype: int64

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [64]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [65]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,accessory,blazerscoatsjackets,onepiece,piece,shoe,sweater,sweatshirthoodie,attribute_value
0,accessory,blazerscoatsjackets,onepiece,piece,shoe,,,"accessory, blazerscoatsjackets, onepiece, piec..."
1,accessory,blazerscoatsjackets,onepiece,piece,shoe,,,"accessory, blazerscoatsjackets, onepiece, piec..."
2,accessory,blazerscoatsjackets,onepiece,piece,shoe,,,"accessory, blazerscoatsjackets, onepiece, piec..."
3,accessory,blazerscoatsjackets,onepiece,piece,shoe,sweater,,"accessory, blazerscoatsjackets, onepiece, piec..."
4,accessory,blazerscoatsjackets,onepiece,piece,shoe,,,"accessory, blazerscoatsjackets, onepiece, piec..."


In [66]:
results_df['attribute_value'].value_counts()

accessory, blazerscoatsjackets, onepiece, piece, shoe             3575
accessory, blazerscoatsjackets, onepiece, piece, shoe, sweater     486
accessory, blazerscoatsjackets, onepiece, shoe                      79
accessory, blazerscoatsjackets, onepiece, shoe, sweater             27
accessory, blazerscoatsjackets, shoe                                 9
accessory, onepiece, shoe                                            6
accessory, onepiece, piece, shoe, sweater                            5
accessory, blazerscoatsjackets, shoe, sweater                        3
accessory, onepiece, piece, shoe                                     2
accessory, onepiece, shoe, sweater                                   2
Name: attribute_value, dtype: int64

In [73]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [0]:
def get_category(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [0]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [71]:
get_category(brand, product_full_name, description, details, brand_category)

EOFError: ignored